In [99]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import cv2
from sklearn.metrics import classification_report,accuracy_score
from utils import DeepSortMock, drawDetection, random_bbox
from sklearn.exceptions import NotFittedError
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn import svm

In [100]:
class Identity():
    def __init__(self, ID, memory_length = 45): # Frames):
        self.ID = ID
        self.age = 0
        self.name = str(ID) 
        self.memory_length = memory_length
        self.__reference_images = []
    def push_image(self,frame):
        self.__reference_images.append(frame)
        self.__reference_images = self.__reference_images[-self.memory_length:]
    def get_images(self):
        return self.__reference_images
    def get_latest_image(self):
        return self.__reference_images[-1]

from skimage.metrics import structural_similarity
import cv2

#Works well with images of different dimensions
def orb_sim(img1, img2):
  # SIFT is no longer available in cv2 so using ORB
  orb = cv2.ORB_create()

  # detect keypoints and descriptors
  kp_a, desc_a = orb.detectAndCompute(img1, None)
  kp_b, desc_b = orb.detectAndCompute(img2, None)

  # define the bruteforce matcher object
  bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
  #perform matches. 
  matches = bf.match(desc_a, desc_b)
  #Look for similar regions with distance < 50. Goes from 0 to 100 so pick a number between.
  similar_regions = [i for i in matches if i.distance < 50]  
  if len(matches) == 0:
    return 0
  return len(similar_regions) / len(matches)

In [ ]:
dim = 128
ppc = 16

re_training_gap = 50 # Frames
grab_local_positive_gap = 10 # Frames
grab_global_negative_gap = 5 # Frames

data_files = ["grandma_A", "grandma_B"]
feeds = [cv2.VideoCapture(f"raw_data/videos/{file}.mp4") for file in data_files]
detectors = [DeepSortMock(f"raw_data/tracks/{file}.pb") for file in data_files]
seperators = [SVC(kernel="poly", probability = True) for _ in data_files]


negative_identity = Identity(0)
negative_identity.memory_length = 250

feed_identities = [dict() for _ in feeds] # TODO: Store in a queue instead
associations = [dict() for _ in feeds]

# DEBUG 
output_frames = []

frames_left = True
frame_num = 0 
try:
    while frames_left:
        frame_num += 1
        output_frames = [ ]
        for i, (feed, detector, seperator, identities) in enumerate(zip(feeds, detectors, seperators, feed_identities)):
            updated_ids = set()
            frames_left, frame = feed.read()
            if not frames_left: break
        
#             # Retrain the SVMs Based on recent image captures
#             if frame_num % re_training_gap == 0:
#                 ppc = 16
#                 hog_features = []
#                 det_ids = []
                
#                 for ID, person in identities.items():
#                     for image in person.get_images():
#                         fd = hog(image, 
#                                 orientations=8, pixels_per_cell=(ppc,ppc),
#                                 cells_per_block=(4, 4),block_norm= 'L2',
#                                 multichannel = True)
        
#                         hog_features.append(fd)
#                         det_ids.append(ID)

#                 if len(identities) > 1:
#                     seperator.fit(hog_features, det_ids)
#                     print("Score", seperator.score(hog_features,det_ids))
                    
#             # Grab Random Non Person image to train seperators  
#             if frame_num % grab_global_negative_gap == 0: 
#                 for _ in range(5):
#                     pass
#                 p1, p2 = random_bbox(frame.shape[0], frame.shape[1], 500, 500)
#                 crop = frame[p1[1]:p2[1], p1[0]:p2[0]]
                
#                 crop = cv2.resize(crop, (dim, dim), interpolation = cv2.INTER_AREA)  
#                 negative_identity.push_image(crop)
                      
            # Query Deep Sort Algorithm
            people = detector.update(frame)
            
            # Grab Positive Person images to train seperators  
            for person in people:
                p1, p2, cat, ID = person
                updated_ids.add(ID)
                if ID not in identities:
                    identities[ID] = Identity(ID)
                frame = drawDetection(frame, (p1, p2, cat, ID), info = identities[ID].name)
                
                crop = frame[p1[1]:p2[1], abs(p1[0]):p2[0]]
                crop = cv2.resize(crop, (dim, dim), interpolation = cv2.INTER_AREA)
                
                identities[ID].name = ""
                identities[ID].push_image(crop)    
            output_frames.append(frame) 
            
            # TODO: Blah Notes
            for identities in feed_identities:
                for identity in identities:
                    if identity in updated_ids:
                        identities[identity].age = 0
                    else: 
                        identities[identity].age += 1
        ##################################################
        ids = []
        for i, identities in enumerate(feed_identities):
                for identity in identities:
                    ids.append((i,identity))
        # Match New Local Identities with Old Global Identities
        scores = [] 
        seen = set()
        for feedA, IdA in ids:
            seen.add((feedA, IdA))
            img_A = feed_identities[feedA][IdA].get_latest_image()
            best_comparison = (0, 0, 0) #Feed
            for feedB, IdB in ids:
                score = 0
                for img_B in feed_identities[feedB][IdB].get_images():
                    score += orb_sim(img_A, img_B)
                score /= len(feed_identities[feedB][IdB].get_images())
                if score > best_comparison[2]:
                    best_comparison = (feedB, IdB, score)
                #if score >= .5:
            print(f"Feed: {feedA}, Main: {IdA}, Best Comp: {best_comparison[1]} Score: {score}")
                # if (feedB, IdB.ID) in seen: continue
                
        cv2.imshow("Joined Product",
            np.concatenate(tuple(output_frame for output_frame in output_frames), axis = 1))
        cv2.waitKey(1)
                
finally:
    [cap.release() for cap in feeds]
    cv2.destroyAllWindows()

Feed: 1, Main: 1, Best Comp: 1 Score: 1.0
Feed: 1, Main: 1, Best Comp: 1 Score: 1.0
Feed: 1, Main: 1, Best Comp: 1 Score: 1.0
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9750784929356358
Feed: 1, Main: 1, Best Comp: 1 Score: 0.978289216025065
Feed: 1, Main: 1, Best Comp: 1 Score: 0.985625256644436
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9675145975903128
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9801891274507599
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9752000483254394
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9839919000048096
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9843548046780805
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9850020863991923
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9909069880365671
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9734301756535098
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9690847829413711
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9834454723312265
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9784540735361769
Feed: 1, Main: 1, Best Comp: 1 Score: 0.9835010678065679
Feed: 1, Main: 1, Bes

In [ ]:
plt.imshow(feed_identities[1][10].get_images()[7])
plt.show()